# Introduction

This notebook analysis the assets in the dataset. The goal is to get an overview of all assets. This will help the analysis of attacks later

# Notebook setup

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 0)


In [3]:
from splunkconnector import core
# from https://github.com/ogrodas/splunk_connector

s=core.SplunkConnect("https://localhost:8089")
s.auth("admin","superadmin")
SPL=s.read_pandas

# Domain controllers

In [10]:
dc=SPL("""
index=botsv2 sourcetype=ActiveDirectory 
| stats earliest(_time) as first,latest(_time) as last,count by dcName
| eval first=strftime(first,"%d/%m/%y %H:%M:%S"), last=strftime(last,"%d/%m/%y %H:%M:%S")
""")
dc

,dcName,first,last,count
0,mercury.frothly.local,19/08/17 05:14:43,29/08/17 10:14:58,13


## Windows assets

In [17]:
winassets=SPL("""
index=botsv2 sourcetype=wineventlog 
| stats earliest(_time) as first,latest(_time) as last,count by ComputerName
| eval first=strftime(first,"%d/%m/%y %H:%M:%S"), last=strftime(last,"%d/%m/%y %H:%M:%S")
| sort -count
""")
winassets 

,ComputerName,first,last,count
0,mercury.frothly.local,03/08/17 18:02:33,29/08/17 11:11:37,314159
1,venus.frothly.local,03/08/17 18:02:08,29/08/17 11:10:48,19537
2,wrk-aturing.frothly.local,03/08/17 18:03:27,29/08/17 11:10:56,13917
3,wrk-btun.frothly.local,03/08/17 18:02:53,29/08/17 11:11:17,13707
4,wrk-ghoppy.frothly.local,03/08/17 18:01:26,29/08/17 11:11:10,13331
5,wrk-fmaltes.frothly.local,03/08/17 18:02:44,29/08/17 11:11:25,13254
6,wrk-abungst.frothly.local,03/08/17 18:00:07,29/08/17 11:11:28,13250
7,wrk-bgist.frothly.local,03/08/17 18:02:47,29/08/17 11:11:18,13078
8,wrk-klagerf.frothly.local,03/08/17 18:02:52,29/08/17 11:11:38,12325
9,WRK-ATURING,27/08/17 23:25:52,27/08/17 23:25:52,1


In [12]:
len(winasset)

10

Mapping windows hostnames to IP-adreses

In [5]:
win_ip_host=SPL("""
    index=botsv2 sourcetype=wineventlog Workstation_Name=* src_ip=10.0.0.0/8 
    | stats earliest(_time) as first ,latest(_time) as last,count by src_ip, Workstation_Name 
    | eval first=strftime(first,"%d/%m/%y %H:%M:%S"), last=strftime(last,"%d/%m/%y %H:%M:%S") 
    | sort -src_ip
""")
win_ip_host

,src_ip,Workstation_Name,first,last,count
0,10.0.4.4,MACLORY-AIR13S,18/08/17 21:27:35,18/08/17 21:27:35,1
1,10.0.4.1,MERCURY,16/08/17 15:30:47,16/08/17 15:30:47,2
2,10.0.2.109,WRK-KLAGERF,03/08/17 18:03:02,29/08/17 11:01:56,263
3,10.0.2.107,WRK-BTUN,24/08/17 03:33:52,26/08/17 08:18:46,50
4,10.0.2.101,WRK-ATURING,29/08/17 10:21:58,29/08/17 10:21:58,1
5,10.0.1.222,MERCURY,08/08/17 13:54:42,29/08/17 10:25:34,130
6,10.0.1.220,MERCURY,03/08/17 18:16:17,29/08/17 10:17:14,109
7,10.0.1.101,VENUS,24/08/17 04:10:54,24/08/17 04:10:54,1
8,10.0.1.100,MERCURY,03/08/17 18:02:36,29/08/17 11:11:33,3162
9,10.0.1.1,GROWLER,03/08/17 18:02:33,29/08/17 11:11:36,68652


# Linux assets

In [16]:
linuxassets=SPL("""
index=botsv2 sourcetype=Unix*| stats earliest(_time) as first,latest(_time) as last,count by host
| eval first=strftime(first,"%d/%m/%y %H:%M:%S"), last=strftime(last,"%d/%m/%y %H:%M:%S")
| sort -count
""")
linuxassets

,host,first,last,count
0,MACLORY-AIR13,18/08/17 21:32:38,29/08/17 10:34:17,2536
1,kutekitten,18/08/17 21:25:31,29/08/17 10:30:52,1392
2,gacrux,08/08/17 13:59:59,29/08/17 10:29:59,703
3,eridanus,08/08/17 14:00:39,29/08/17 10:30:38,676


In [18]:
len(linuxassets)

4

##  src_ip overview

In [20]:
srcip=SPL("""
index=botsv2 | stats count by src_ip | sort -count
""")

In [21]:
srcip[:10]

,src_ip,count
0,10.0.1.1,387840
1,10.0.1.100,312601
2,10.0.2.109,184068
3,10.0.2.107,165859
4,172.31.10.10,142242
5,10.0.4.4,133092
6,111.249.38.34,104984
7,118.166.53.142,84086
8,10.0.1.200,82906
9,52.40.10.231,80649


In [22]:
len(srcip)

1878

## DNS-IP mapping


todo:
index=botsv2  dns | spath "message_type{}" | search "message_type{}"=RESPONSE| spath "host_addr{}" | search "host_addr{}"=10.0.0.0/8 | table host_addr{},name{}